In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras,gc,time
from keras.layers import *
from keras.models import Sequential,Model

### Cargar datos

In [4]:
folder = "../proyecto/datasets/"
df_independencia_verano = pd.read_csv(folder+"dump-Independencia_2018-04-12_230000-verano.csv")
df_independencia_invierno = pd.read_csv(folder+"dump-Independencia_2018-04-12_230000-invierno.csv")

df_condes_verano = pd.read_csv(folder+"dump-Las_Condes_2018-04-12_230000-verano.csv")
df_condes_invierno = pd.read_csv(folder+"dump-Las_Condes_2018-04-12_230000-invierno.csv")

df_independencia_verano.head()

,registered_on,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
0,1997-11-01 01:00:00,2.7,63.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1997-11-01 02:00:00,2.6,54.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,1997-11-01 03:00:00,2.2,53.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,1997-11-01 04:00:00,2.5,65.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,1997-11-01 05:00:00,2.3,118.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [5]:
df_independencia_verano.tail()

,registered_on,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
75081,2018-03-31 19:00:00,NaN,40.0,9.0,NaN,NaN,NaN,NaN,164.584,43.0000,21.5583,1.245590,NaN,0.0,0.0,6.86500
75082,2018-03-31 20:00:00,NaN,33.5,9.0,NaN,NaN,NaN,NaN,169.048,50.0000,19.5583,1.169250,NaN,0.0,0.0,5.95750
75083,2018-03-31 21:00:00,NaN,24.5,11.0,NaN,NaN,NaN,NaN,207.717,54.5833,17.8083,0.966588,NaN,0.0,0.0,8.92751
75084,2018-03-31 22:00:00,NaN,27.5,10.0,NaN,NaN,NaN,NaN,189.509,58.4167,16.6667,0.816365,NaN,0.0,0.0,4.88500
75085,2018-03-31 23:00:00,NaN,29.0,10.0,NaN,NaN,NaN,NaN,231.942,61.4167,15.9083,1.116220,NaN,0.0,0.0,2.41000


In [6]:
## ultimo año como test
df_condes_verano_train = df_condes_verano[(df_condes_verano["registered_on"]<"2017")]
df_condes_invierno_train = df_condes_invierno[(df_condes_invierno["registered_on"]<"2017")]
df_condes_verano_test = df_condes_verano[df_condes_verano["registered_on"]>="2017"]
df_condes_invierno_test = df_condes_invierno[df_condes_invierno["registered_on"]>="2017"]

df_condes_verano_train.tail()

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
69390,2016-12-31 14:00:00,NaN,0.553417,53.0,24.0,7.0400,3.2450,10.28500,NaN,NaN,41.8333,26.6333,NaN,NaN,NaN,NaN,72.5233
69391,2016-12-31 15:00:00,NaN,0.467584,22.0,21.0,4.4825,2.7500,7.23250,NaN,NaN,39.3333,26.3167,NaN,NaN,NaN,NaN,58.9725
69392,2016-12-31 16:00:00,NaN,0.459000,16.5,13.0,3.4925,2.9975,6.49000,NaN,NaN,37.3333,26.0583,NaN,NaN,NaN,NaN,50.3033
69393,2016-12-31 17:00:00,NaN,0.459000,15.0,14.0,3.9050,3.4100,7.31500,NaN,NaN,36.3333,25.1417,NaN,NaN,NaN,NaN,42.9808
69394,2016-12-31 18:00:00,NaN,0.450417,NaN,NaN,6.6275,3.1625,9.79001,NaN,NaN,38.8333,24.1750,NaN,NaN,NaN,NaN,NaN


In [6]:
df_condes_verano_test.head()

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
69395,2017-01-03 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,28.834697,NaN
69396,2017-01-03 14:00:00,NaN,NaN,19.5,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,24.715235,NaN
69397,2017-01-03 15:00:00,NaN,0.836667,13.5,18.0,10.2575,1.7600,12.0175,NaN,NaN,14.9167,28.8167,NaN,NaN,0.0,24.537322,73.6176
69398,2017-01-03 16:00:00,NaN,0.768001,19.5,2.0,10.8350,2.5025,13.3375,NaN,NaN,12.2500,28.2917,NaN,NaN,0.0,17.778006,57.6258
69399,2017-01-03 17:00:00,NaN,0.793750,72.0,10.0,11.5775,2.5850,14.1625,NaN,NaN,14.5000,27.4250,NaN,NaN,0.0,9.862870,51.9867


In [7]:
df_condes_verano.dropna(how='any')

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3


Si elimino todas las que tengan al menos un null se eliminan todos los registros

In [8]:
df_condes_verano_train.loc[:,rows_to_use].dropna(subset=["O3"])#(how="all",O3)

NameError: name 'rows_to_use' is not defined

### PRe PRocesarlos

In [7]:
## representacion
rows_to_use = ["CO","PM10","PM25","NO2","NO","NOX","SO2","WD","RH","TEMP","WS","HCNM","UVA","UVB","O3"]


def create_sequences(dataframe,lag=1,dia=True):
    #preprocesamiento asumiendo dataset es una secuencia
    timestep = dataframe.loc[:,"registered_on"].values
    
    if dia:
        indices_orden = []
        dia = timestep[0].split("-")[-1].split(" ")[0]
        aux_indices1 = 0
        aux_indices2 = 0 
        for dato in timestep[1:]:
            nuevo_dia = dato.split("-")[-1].split(" ")[0]
            aux_indices2+=1
            if dia != nuevo_dia:
                dia = nuevo_dia
                indices_orden.append([aux_indices1,aux_indices2])
                aux_indices1 = aux_indices2
        indices_orden.append([aux_indices1,aux_indices2+1])
        
        aux =  []
        data = dataframe.loc[:,rows_to_use].values
        for init,finit in indices_orden:
            aux+=[np.nanmax(data[init:finit],axis=0)]
        data = np.asarray(aux)
        #dropnan... de O3
        data = data[~np.isnan(data)[:,-1]]
    else:
        #dropnan... de o3
        data = dataframe.loc[:,rows_to_use].dropna(subset=["O3"]).values
    
    #rellenar nans de valores nada?
   # data = np.asarray([[-1. if np.isnan(x) else x for x in array] for array in data])
    
    dataX = []
    dataY =[]
    for t_plus_1 in range(lag,len(data)):
        #se crea el Y (target)
        predecir = data[t_plus_1][-1] #ultima columna es O3
        dataY.append(predecir)
        #se crea el X (inputs) columnas para predecir Y
        columnasX = []
        for i in np.arange(lag,0,-1): #para los valores anteriores al t_plus_1 durante un lag
            columnasX.append( data[t_plus_1-i] )
        dataX.append(columnasX)
    return np.array(dataX),np.array(dataY).reshape(len(dataY),1)

In [8]:
lag = 3
dia=True
trainX1, trainY1 = create_sequences(df_condes_verano_train, lag,dia)
trainX2, trainY2 = create_sequences(df_condes_invierno_train, lag,dia)

testX1, testY1 = create_sequences(df_condes_verano_test, lag,dia)
testX2, testY2 = create_sequences(df_condes_invierno_test, lag,dia)

print("All arrays shapes")
print("Train verano (input): ",trainX1.shape)
print("Train invierno (input): ",trainX2.shape)
print("Train verano (output): ",trainY1.shape)
print("Train invierno (output): ",trainY2.shape)

/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:26: RuntimeWarning: All-NaN slice encountered


All arrays shapes
Train verano (input):  (2931, 3, 15)
Train invierno (input):  (2401, 3, 15)
Train verano (output):  (2931, 1)
Train invierno (output):  (2401, 1)


In [10]:
def trid_to_2d(array,backward=False):
    if backward:
        return array.reshape(array.shape[0]/lag, lag ,array.shape[-1])
    else:
        return array.reshape((np.prod(array.shape[:2]), array.shape[-1]))

In [11]:
trid_to_2d(trid_to_2d(trainX),backward=True).shape

NameError: name 'trainX' is not defined

### Escalar datos y transformar para entrada

In [9]:
#calculo de estadisticos ("fit") MinMaxScaler
class MinMax(object):
    def __init__(self,aux=True):
        self.minimos = None
        self.maximos = None
        
    def fit(self,data):
        self.minimos = np.nanmin(data, axis=(0,1))#, keepdims=True)
        self.maximos = np.nanmax(data ,axis=(0,1))#, keepdims=True)
    
    def transform(self,data):
        #normalize between 0 and 1 every statisctic
        return (data-self.minimos) / (self.maximos-self.minimos)
    
    def inverse_transform(self,data):
        return data*(self.maximos-self.minimos) + self.minimos
    
scaler_model1 = MinMax()
scaler_model1.fit(trainX1)
X_train1 = scaler_model1.transform(trainX1)
X_test1 = scaler_model1.transform(testX1)

scaler_model2 = MinMax()
scaler_model2.fit(trainX2)
X_train2 = scaler_model2.transform(trainX2)
X_test2 = scaler_model2.transform(testX2)

X_train1.shape

(2931, 3, 15)

In [10]:
print("Minimos en train: ", np.nanmin(X_train1, axis=(0,1)))
print("Minimos en test: ", np.nanmin(X_test1, axis=(0,1)))

Minimos en train:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Minimos en test:  [6.89655125e-08 2.74869110e-02 2.99625468e-02 1.12001868e-01
 4.98162247e-03 2.49952114e-02            nan            nan
 2.50000000e-01 8.71331507e-02            nan            nan
 0.00000000e+00 5.81265886e-04 8.90017964e-02]


/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: All-NaN slice encountered
  from ipykernel import kernelapp as app


In [11]:
print("Maximos en train: ", np.nanmax(X_train2, axis=(0,1)))
print("Maximos en test: ", np.nanmax(X_test2, axis=(0,1)))

Maximos en train:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Maximos en test:  [0.25981294 0.34102142 0.58563536 0.93545451 0.74557729 0.70256368
        nan        nan 0.91729421 0.85851963        nan        nan
 0.04044388 0.24578276 0.37012987]


/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: All-NaN slice encountered
  from ipykernel import kernelapp as app


In [12]:
mask_value = -1.
def nan_to_num(data,mask_value):
    return np.asarray([[[mask_value if np.isnan(x) else x for x in timestep] for timestep in array] for array in data])

X_train1 = nan_to_num(X_train1,mask_value)
X_test1 = nan_to_num(X_test1,mask_value)
X_train2 = nan_to_num(X_train2,mask_value)
X_test2 = nan_to_num(X_test2,mask_value)
print("Mascara agregada")

Mascara agregada


In [16]:
#para el Y tambien...
from sklearn.preprocessing import MinMaxScaler
#o standar
from sklearn.preprocessing import StandardScaler

scaler_model = MinMaxScaler()
scaler_model.fit(trainY)
y_train = scaler_model.transform(trainY)
y_test = scaler_model.transform(testY)
y_train.shape

NameError: name 'trainY' is not defined

Características del modelo:
* Predecir el máximo de ozono $O_3$ del día siguiente (del período de 24 horas).
* Grano de predicción: Horas o día?
* Cuanta información anterior utilizar? ..3 dias
* Nulos? en $O_3$ o contaminantes?
    * Borrar
    * Mascara

In [ ]:
#predecir el máximo del dia siguiente...
#

## Modelo

In [13]:
batch_size = 32
epochs = 15
opt = 'adam'
loss_utilizada = 'mse' #?
nhidden = 64
from attention import *

In [136]:
modelo1 = Sequential()
#modelo1.add(LSTM(nhidden, input_shape=(timesteps, features), return_sequences=True))
modelo1.add(AttentionDecoder(nhidden, nhidden,input_shape=(timesteps,features),return_sequences=False)) #que son esos  (units, output_dim)

#modelo1.add(LSTM(nhidden, return_sequences=False))

modelo1.add(Dense(1,activation='linear')) #1 prediccion?
modelo1.compile(loss=loss_utilizada, optimizer=opt, metrics=['acc'])
modelo1.summary()
# train LSTM

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
AttentionDecoder (AttentionD (None, 3, 64)             43008     
_________________________________________________________________
dense_12 (Dense)             (None, 3, 1)              65        
Total params: 43,073
Trainable params: 43,073
Non-trainable params: 0
_________________________________________________________________


In [14]:
#https://github.com/farizrahman4u/seq2seq
import seq2seq
from seq2seq.models import Seq2Seq

model = Seq2Seq(input_dim=5, hidden_dim=10, output_length=8, output_dim=8)
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_91 (InputLayer)           (None, None, 5)      0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 10)     60          input_91[0][0]                   
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__o

In [32]:
from seq2seq.models import AttentionSeq2Seq

model = AttentionSeq2Seq(input_dim=features, input_length=timesteps, hidden_dim=10, output_length=timesteps, output_dim=20, depth=1)
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_725 (InputLayer)          (None, 3, 15)        0                                            
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 3, 10)        2080        input_725[0][0]                  
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
recurrent_

In [33]:
bla = Sequential()
bla.add(Masking(mask_value=mask_value, input_shape=(timesteps, features)))
bla.layers[-1].output

<tf.Tensor 'masking_13/mul:0' shape=(?, 3, 15) dtype=float32>

In [46]:
auuuux = Model(inputs=model.layers[1].input,outputs=model.outputs)
auuuux.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_725 (InputLayer)          (None, 3, 15)        0                                            
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 3, 10)        2080        input_725[0][0]                  
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
recurrent_

In [39]:
auuuux(model.layers[0].input)

<tf.Tensor 'model_236_1/recurrent_sequential_20/transpose_1:0' shape=(?, ?, 20) dtype=float32>

In [47]:
auuuux(bla.output)

ValueError: Shapes must be equal rank, but are 2 and 3 for 'model_239/recurrent_sequential_20/while/Select' (op: 'Select') with input shapes: [?,?], [?,20], [?,?,10].

In [22]:
chikilin = Model(inputs=bla.inputs,outputs=bla.outputs)
chikilin.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_10_input (InputLayer (None, 3, 15)             0         
_________________________________________________________________
masking_10 (Masking)         (None, 3, 15)             0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [49]:
timesteps = X_train1.shape[1]
features = X_train1.shape[2] #todo excepto o3

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def attention_multiply(vects):
    encoder, attention = vects
    return K.batch_dot(attention,encoder, axes=1)

def modelo(tipo,dia=True):
    model = Sequential()
    model.add(Masking(mask_value=mask_value, input_shape=(timesteps, features)))

    if tipo=="sin atencion":
        model.add(GRU(nhidden,return_sequences=True))
        model.add(GRU(nhidden,return_sequences=True))
    elif tipo=="atencion":
        model.add(GRU(nhidden, return_sequences=True))
        model.add(AttentionDecoder(nhidden, nhidden)) #que son esos  (units, output_dim)
    
    elif tipo=="sin atencion libreria":
        aux_model = Seq2Seq(input_dim=features, hidden_dim=nhidden, output_length=timesteps, output_dim=nhidden,depth=1)
        #teacherforse=True
        #aux_model = Model(inputs=model.inputs,outputs=aux_model(model.outputs))
        
        layer_aux =  GRU(nhidden, return_sequences=False)(aux_model.outputs)
        layer_aux = Dense(1,activation='linear')(layer_aux) #1 prediccion?
        
        model = Model(inputs=aux_model.inputs,outputs=layer_aux)
        model.compile(loss=loss_utilizada,optimizer=opt)
        model.summary()
        return model
    
    elif tipo=="atencion libreria":
        aux_model = AttentionSeq2Seq(input_dim=features,input_length=timesteps,hidden_dim=nhidden
                                     ,output_length=timesteps,output_dim=nhidden,depth=1)
        model = Sequential()
        model.add(aux_model)
    
    if dia:
        model.add(GRU(nhidden, return_sequences=False))
        model.add(Dense(1,activation='linear')) #1 prediccion?
    else:
        model.add(TimeDistributed(Dense(1,activation='linear'))) #varias prediccions--a nivel de horario..
        model.add(GlobalMaxPooling1D())
    
    model.compile(loss=loss_utilizada,optimizer=opt)
    model.summary()
    return model

modelo1 = modelo("sin atencion",dia)
modelo2 = modelo("atencion",dia)
modelo3 = modelo("sin atencion libreria",dia)
modelo4 = modelo("atencion libreria",dia)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_17 (Masking)         (None, 3, 15)             0         
_________________________________________________________________
gru_21 (GRU)                 (None, 3, 64)             15360     
_________________________________________________________________
gru_22 (GRU)                 (None, 3, 64)             24768     
_________________________________________________________________
gru_23 (GRU)                 (None, 64)                24768     
_________________________________________________________________
dense_482 (Dense)            (None, 1)                 65        
Total params: 64,961
Trainable params: 64,961
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
maskin

In [48]:
modelo1.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))

Train on 2931 samples, validate on 195 samples
Epoch 1/15
2931/2931 [==============================] - 8s 3ms/step - loss: 5679.5794 - val_loss: 3322.8662
Epoch 2/15
2931/2931 [==============================] - 3s 1ms/step - loss: 4446.7375 - val_loss: 2694.6711
Epoch 3/15
2931/2931 [==============================] - 3s 1000us/step - loss: 3744.6682 - val_loss: 2193.5595
Epoch 4/15
2931/2931 [==============================] - 3s 1ms/step - loss: 3159.5775 - val_loss: 1783.9648
Epoch 5/15
2931/2931 [==============================] - 3s 998us/step - loss: 2666.9916 - val_loss: 1451.4994
Epoch 6/15
2931/2931 [==============================] - 3s 1ms/step - loss: 2253.8573 - val_loss: 1183.3077
Epoch 7/15
2931/2931 [==============================] - 3s 997us/step - loss: 1909.0314 - val_loss: 974.3556
Epoch 8/15
2931/2931 [==============================] - 3s 997us/step - loss: 1623.9548 - val_loss: 811.3651
Epoch 9/15
2931/2931 [==============================] - 3s 1ms/step - loss: 1390.5

In [49]:
modelo2.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))

Train on 2931 samples, validate on 195 samples
Epoch 1/15
2931/2931 [==============================] - 9s 3ms/step - loss: 6232.6228 - val_loss: 3216.8350
Epoch 2/15
2931/2931 [==============================] - 4s 1ms/step - loss: 4226.4116 - val_loss: 2479.1321
Epoch 3/15
2931/2931 [==============================] - 4s 1ms/step - loss: 3458.2206 - val_loss: 1965.5802
Epoch 4/15
2931/2931 [==============================] - 4s 1ms/step - loss: 2867.0932 - val_loss: 1568.8552
Epoch 5/15
2931/2931 [==============================] - 4s 1ms/step - loss: 2388.3118 - val_loss: 1259.6303
Epoch 6/15
2931/2931 [==============================] - 4s 1ms/step - loss: 1998.0834 - val_loss: 1019.3847
Epoch 7/15
2931/2931 [==============================] - 4s 1ms/step - loss: 1680.8844 - val_loss: 838.3726
Epoch 8/15
2931/2931 [==============================] - 4s 1ms/step - loss: 1424.3935 - val_loss: 704.2349
Epoch 9/15
2931/2931 [==============================] - 4s 1ms/step - loss: 1220.1080 - val

In [52]:
modelo3.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))

Train on 2931 samples, validate on 195 samples
Epoch 1/15
2931/2931 [==============================] - 22s 8ms/step - loss: 5651.3490 - val_loss: 3342.4192
Epoch 2/15
2931/2931 [==============================] - 3s 899us/step - loss: 4475.7698 - val_loss: 2720.9909
Epoch 3/15
2931/2931 [==============================] - 3s 892us/step - loss: 3776.4309 - val_loss: 2217.7048
Epoch 4/15
2931/2931 [==============================] - 3s 885us/step - loss: 3190.8862 - val_loss: 1806.2612
Epoch 5/15
2931/2931 [==============================] - 3s 895us/step - loss: 2697.2063 - val_loss: 1472.0465
Epoch 6/15
2931/2931 [==============================] - 3s 885us/step - loss: 2281.5682 - val_loss: 1201.0886
Epoch 7/15
2931/2931 [==============================] - 3s 890us/step - loss: 1933.8492 - val_loss: 988.2500
Epoch 8/15
2931/2931 [==============================] - 3s 891us/step - loss: 1645.8529 - val_loss: 823.1851
Epoch 9/15
2931/2931 [==============================] - 3s 883us/step - loss

In [53]:
modelo4.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))

Train on 2931 samples, validate on 195 samples
Epoch 1/15
2931/2931 [==============================] - 13s 4ms/step - loss: 5641.3066 - val_loss: 3279.1513
Epoch 2/15
2931/2931 [==============================] - 3s 1ms/step - loss: 4395.8773 - val_loss: 2658.1465
Epoch 3/15
2931/2931 [==============================] - 3s 1ms/step - loss: 3699.4842 - val_loss: 2159.9836
Epoch 4/15
2931/2931 [==============================] - 3s 1ms/step - loss: 3119.7706 - val_loss: 1755.7045
Epoch 5/15
2931/2931 [==============================] - 3s 1ms/step - loss: 2632.7764 - val_loss: 1427.1046
Epoch 6/15
2931/2931 [==============================] - 3s 1ms/step - loss: 2224.0965 - val_loss: 1164.2534
Epoch 7/15
2931/2931 [==============================] - 3s 1ms/step - loss: 1883.1651 - val_loss: 957.8679
Epoch 8/15
2931/2931 [==============================] - 3s 1ms/step - loss: 1602.1872 - val_loss: 798.1491
Epoch 9/15
2931/2931 [==============================] - 3s 1ms/step - loss: 1372.0597 - va

In [84]:
#ivnierno
modelo1.fit(X_train2, trainY2, epochs=epochs, batch_size=batch_size, validation_data=(X_test2, testY2))
modelo2.fit(X_train2, trainY2, epochs=epochs, batch_size=batch_size, validation_data=(X_test2, testY2))

Train on 2401 samples, validate on 116 samples
Epoch 1/15
2401/2401 [==============================] - 6s 3ms/step - loss: 995.3553 - val_loss: 91.2107
Epoch 2/15
2401/2401 [==============================] - 2s 807us/step - loss: 584.0577 - val_loss: 81.9976
Epoch 3/15
2401/2401 [==============================] - 2s 805us/step - loss: 497.9957 - val_loss: 104.9367
Epoch 4/15
2401/2401 [==============================] - 2s 819us/step - loss: 459.7946 - val_loss: 135.8880
Epoch 5/15
2401/2401 [==============================] - 2s 810us/step - loss: 443.6389 - val_loss: 160.3402
Epoch 6/15
2401/2401 [==============================] - 2s 811us/step - loss: 437.5684 - val_loss: 178.2333
Epoch 7/15
2401/2401 [==============================] - 2s 796us/step - loss: 435.0719 - val_loss: 191.1585
Epoch 8/15
2401/2401 [==============================] - 2s 827us/step - loss: 433.9959 - val_loss: 201.6697
Epoch 9/15
2401/2401 [==============================] - 2s 800us/step - loss: 433.6821 - val_

In [9]:
## evaluación?
rmse?



#if es hora.. max
# si es por dia ya predice max

In [50]:
def evaluar_modelo(modelito,data):
    mse_test = modelito.evaluate(data[0],data[1])
    print("RMSE del modelo en test: ",np.sqrt(mse_test))

In [51]:
evaluar_modelo(modelo1,[X_test1,testY1])   

195/195 [==============================] - 0s 455us/step
('RMSE del modelo en test: ', 22.082310224584013)


In [52]:
evaluar_modelo(modelo2,[X_test1,testY1])      

195/195 [==============================] - 0s 456us/step
('RMSE del modelo en test: ', 22.356093350644233)
